# NOTE IMPORTANTI
Considerato che:
- sembra funzionare meglio l'interpolante lineare per interpolare il cmd.
- sembra funzionare meglio l'interpolazione sui valori medi di quella sui valori mediani

concluderei che **usiamo l'interpolante lineare applicata ai valori medi**.

In [112]:
import numpy as np
import scipy.interpolate as interp
import matplotlib
#matplotlib.use('NBAgg')
#matplotlib.use('wxAgg')
%matplotlib wx
from matplotlib import pyplot as plt
from cursor import Cursor

# CARICAMENTO DEI DATI E SETUP INIZIALE

In [2]:
datacolnames = ['x','y','v','sv','i','si','sel']
data = np.genfromtxt('./NGC6121.cal.xym',
                     skip_header=2,
                     names=datacolnames)
print data.shape

(9666L,)


In [3]:
# estraggo il vettore delle selezioni
sel = np.asarray(data['sel'],dtype='int') == True
# tengo solo le stelle "buone", estraggo mag in v ed i, quindi calcolo v-i
data_good = data[:][sel]
print data_good.shape

(8503L,)


In [143]:
# per comodità assegno le colonne rilevanti a variabili differenti
v = data_good['v']
sv = data_good['sv']
i = data_good['i']
si = data_good['si']
v_i = v-i
sv_i = np.sqrt(sv**2+si**2)

In [153]:
# cella di test, NON ESEGUIRE.
plt.figure()
plt.plot(v_i,v,'.',markersize=2)
plt.errorbar(v_i,v,xerr=sv_i,yerr=sv,fmt='none')
plt.gca().invert_yaxis()
plt.show()

### La cella che segue va lanciata ogni volta che la figura viene chiusa!
Non è invece necessario lanciare tutte quelle che seguono, basta ritornare alle celle dei plot per visualizzarli sovrapposti al cmd.

In [113]:
# plotto il cmd
plt.figure(figsize=(9,7))
cmd_plot, = plt.plot(v_i,v,'k.',markersize=2,label='NGC6121',alpha=0.5)
# inverto l'asse y
plt.gca().invert_yaxis()
# ottengo figura e assi correnti
fig = plt.gcf()
ax = plt.gca()
# aggiungo le label agli assi ed aggiungo la legenda
ax.set_xlabel('v-i',fontsize=14)
ax.set_ylabel('v',fontsize=14)
plt.legend(fontsize=13)
# mostro il grafico
plt.show()

# 1) CERCO ETA' CON ISOCRONE

### Seleziono le stelle nel MS, TO ed RGB
**ATTENZIONE!!!!** Se sono presenti i file con i limiti in colore al cmd si saltano le celle seguenti e li si caricano.

In [ ]:
# seleziono il limite sinistro di MS, TO e RGB
left_limit = Cursor(fig,ax,color='c',zoom=[1.,2.])
left_limit.start()

In [ ]:
# seleziono il limite destro di MS, TO e RGB
right_limit = Cursor(fig,ax,color='c',zoom=[1.,2.])
right_limit.start()

In [ ]:
# estraggo tutti i punti destri e sinistri
xl, yl = left_limit.get_x_y()
#print xl
#print yl
xr, yr = right_limit.get_x_y()
#print xr
#print yr

In [ ]:
# salvo i punti selezionati per usi futuri, anche in caso di riavvio del kernel
# TOGLIERE I COMMENTI SOLO SE MOOOOLTO SICURI DI QUELLO CHE SI STA FACENDO
#np.savetxt('cmd_selection_left.dat',np.column_stack((xl,yl)),header='xl \t yl')
#np.savetxt('cmd_selection_right.dat',np.column_stack((xr,yr)),header='xr \t yr')

In [ ]:
# carico i punti selezionati SOLO SE SERVE
l = np.genfromtxt('cmd_selection_left.dat',skip_header=1)
xl = l[:,0]
yl = l[:,1]
r = np.genfromtxt('cmd_selection_right.dat',skip_header=1)
xr = r[:,0]
yr = r[:,0]
# plotto i punti SE NECESSARIO
plt.plot(xl,yl,'c.')
plt.plot(xr,yr,'c.')

In [ ]:
# costruisco l'interpolatore con i punti ottenuti con il cursore
interp_left = interp.interp1d(yl,xl,bounds_error=False,fill_value='extrapolate')
interp_right = interp.interp1d(yr,xr,bounds_error=False,fill_value='extrapolate')
# mi preparo ad interpolare ordinando v ed ordino anche v_i in modo che segua v
index_sorted = v.argsort()
v_sorted = v[index_sorted]
v_i_sorted = v_i[index_sorted]
# ottengo i valori interpolati di v-i usando v
v_i_left = interp_left(np.sort(v))
v_i_right = interp_right(np.sort(v))

In [ ]:
# salvo i valori di v-i interpolati
# TOGLIERE I COMMENTI SOLO SE MOOOOLTO SICURI DI QUELLO CHE SI STA FACENDO
#np.savetxt('cmd_bound.dat',np.column_stack((v_i_left,v_i_right)),header='left_boundary \t right_boundary')

**Carico i limiti (in colore) al cmd se precedentemente salvati.**

In [5]:
# carico i limiti al cmd
bound = np.genfromtxt('cmd_bound.dat',skip_header=1)
# definisco gli array appositi
v_i_left = bound[:,0]
v_i_right = bound[:,1]
# ordino questi array
index_sorted = v.argsort()
v_sorted = v[index_sorted]
v_i_sorted = v_i[index_sorted]

In [ ]:
# plotto i punti interpolati sui valori di v [limite destro e sinistro]
# plt.figure()
plt.plot(v_i_left,np.sort(v),'r.',markersize=1)
plt.plot(v_i_right,np.sort(v),'r.',markersize=1)
plt.show()

In [6]:
# seleziono tutti i punti entro il limite destro e sinistro
cond_restrict = np.where((v_i_sorted>v_i_left) & (v_i_sorted<v_i_right))
v_i_restricted = v_i_sorted[cond_restrict]
v_restricted = v_sorted[cond_restrict]

In [ ]:
# plotto i punti interni ai limiti
#plt.figure()
cmd_restricted_plot = plt.plot(v_i_restricted,v_restricted,'mo',markersize=4,alpha=0.2,label='points inside selection')
plt.legend(fontsize=13)
plt.show()

In [7]:
# ottengo i valori MEDI del cmd lungo MS, TO e RGB
v_mean = []
v_sigma = []
v_i_mean = []
v_i_sigma = []
mag_bin_mean = 0.1
max_mag_mean = 21
min_mag_mean = 12
bin_interp_mean = 0.01
for mag in np.arange(min_mag_mean, max_mag_mean, mag_bin_mean):
    points_in_bin = np.where((v_restricted>mag) & (v_restricted<mag+mag_bin_mean))
    if points_in_bin[0].shape[0] <1:
        continue
    v_mean.append(np.mean(v_restricted[points_in_bin]))
    v_sigma.append(np.std(v_restricted[points_in_bin]))
    v_i_mean.append(np.mean(v_i_restricted[points_in_bin]))
    v_i_sigma.append(np.std(v_i_restricted[points_in_bin]))

v_mean = np.array(v_mean)
v_sigma = np.array(v_sigma)
v_i_mean = np.array(v_i_mean)
v_i_sigma = np.array(v_i_sigma)

In [134]:
# plotto i valori appena ottenuti
plt.figure()
plt.plot(v_i_mean,v_mean,'r.',markersize=4)
plt.errorbar(v_i_mean,v_mean,xerr=v_i_sigma,yerr=v_sigma)
plt.gca().invert_yaxis()
plt.show()

In [8]:
# interpolo linearmente sui valori medi [APPARENTEMENTE LA SCELTA MIGLIORE]
interp_mean = interp.interp1d(v_mean,v_i_mean,bounds_error=False,fill_value='extrapolate')
v_interp_mean = np.arange(min_mag_mean,max_mag_mean,bin_interp_mean)
v_i_interp_mean = interp_mean(v_interp_mean)

In [128]:
# plotto il risultato dell'interpolazione lineare sui valori medi
#cmd_mean_plot.remove()
cmd_mean_plot, = plt.plot(v_i_interp_mean,v_interp_mean,'r-',label='interp lin su punti medi',alpha=0.6)
cmd_sigma_plot, = plt.errorbar(v_i_interp_mean,v_interp_mean,xerr=v_i_sigma,yerr=v_sigma)
cmd_sigma_area_plot = plt.fill_between(v_i_mean,v_mean-v_sigma,v_mean+v_sigma)
plt.legend()
plt.show()

ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [ ]:
# interpolo con una spline i valori medi [NON USARE]
spline_interp_mean = interp.splrep(v_mean,v_i_mean)
v_spline_mean = np.arange(min_mag_mean,max_mag_mean,bin_interp_mean)
v_i_spline_mean = interp.splev(v_spline_mean,spline_interp_mean)

In [ ]:
# plotto il risultato dell'interpolazione spline sui valori medi [NON USARE]
plt.plot(v_i_spline_mean,v_spline_mean,'r-',label='interp spline su punti medi',alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# ottengo i valori MEDIANI del cmd lungo MS, TO e RGB [NON USARE]
v_median = []
v_percentile68 = []
v_i_median = []
v_i_percentile68 = []
mag_bin_median = 0.05
max_mag_median = 21
min_mag_median = 12
bin_interp_median = 0.01
for mag in np.arange(min_mag_median, max_mag_median, mag_bin_median):
    points_in_bin = np.where((v_restricted>mag) & (v_restricted<mag+mag_bin_median))
    if points_in_bin[0].shape[0] <1:
        continue
    v_median.append(np.median(v_restricted[points_in_bin]))
    v_percentile68.append(np.percentile(68, v_restricted[points_in_bin]))
    v_i_median.append(np.median(v_i_restricted[points_in_bin]))
    v_i_percentile68.append(np.percentile(68, v_i_restricted[points_in_bin]))

v_median = np.array(v_median)
v_percentile68 = np.array(v_percentile68)
v_i_median = np.array(v_i_median)
v_i_percentile68 = np.array(v_i_percentile68)

In [ ]:
# plotto i valori appena ottenuti [NON USARE]
plt.plot(v_i_median,v_median,'g.',markersize=4)
plt.show()

In [ ]:
# interpolo linearmente i valori mediani [NON USARE]
interp_median = interp.interp1d(v_median,v_i_median,bounds_error=False,fill_value='extrapolate')
v_interp_median = np.arange(min_mag_median,max_mag_median,bin_interp_median)
v_i_interp_median = interp_median(v_interp_median)

In [ ]:
# plotto il risultato dell'inteprolazione lineare sui valori mediani [NON USARE]
plt.plot(v_i_interp_median,v_interp_median,'g--',label='interp lin su punti mediani',alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# intrpolo con una spline i valori mediani [NON USARE]
spline_interp_median = interp.splrep(v_median,v_i_median)
v_spline_median = np.arange(min_mag_median,max_mag_median,0.01)
v_i_spline_median = interp.splev(v_spline_median,spline_interp_median)

In [ ]:
# plotto il risultato dell'interpolazione spline sui valori mediani [NON USARE]
plt.plot(v_i_spline_median,v_spline_median,'g-',label='interp spline su punti mediani',alpha=0.6)
plt.legend()
plt.show()

### Confronto con diverse isocrone con gli stessi E(B-V) e mod_V

In [52]:
# carico le isocrone
isocolnames = ['v','i']
iso1 = np.genfromtxt('./iso10.5.dat',
                    skip_header=8,
                    usecols=(7,9),
                    names=isocolnames)
iso2 = np.genfromtxt('./iso11.5.dat',
                    skip_header=8,
                    usecols=(7,9),
                    names=isocolnames)
iso3 = np.genfromtxt('./iso12.0.dat',
                    skip_header=8,
                    usecols=(7,9),
                    names=isocolnames)
iso4 = np.genfromtxt('./iso12.5.dat',
                    skip_header=8,
                    usecols=(7,9),
                    names=isocolnames)

In [10]:
# porto le isocrone al piano osservativo
ebv = (1-0.09)*0.35
print "E(B-V) =", ebv
modv = (1-0.015)*12.8
print "mod_V =", modv
Av = 3.1 * ebv
print "Av =", Av
Ai = 0.479 * Av
print "Ai =", Ai
mod0 = modv - Av
print "mod0 =", mod0
v1 = iso1['v'] + mod0 + Av
i1 = iso1['i'] + mod0 + Ai
v2 = iso2['v'] + mod0 + Av
i2 = iso2['i'] + mod0 + Ai
v3 = iso3['v'] + mod0 + Av
i3 = iso3['i'] + mod0 + Ai
v4 = iso4['v'] + mod0 + Av
i4 = iso4['i'] + mod0 + Ai

E(B-V) = 0.3185
mod_V = 12.608
Av = 0.98735
Ai = 0.47294065
mod0 = 11.62065


In [55]:
# limito le isocrone al range di interesse
sel1 = np.where((v1>12) & (v1<21))
v1good = v1[sel1]
i1good = i1[sel1]
sel2 = np.where((v2>12) & (v2<21))
v2good = v2[sel2]
i2good = i2[sel2]
sel3 = np.where((v3>12) & (v3<21))
v3good = v3[sel3]
i3good = i3[sel3]
sel4 = np.where((v4>12) & (v4<21))
v4good = v4[sel4]
i4good = i4[sel4]

In [57]:
# Questa cella permette di rimuovere TUTTE le isocrone plottate (usare solo dopo aver plottato le isocrone una prima volta)
if 'iso_plot' in locals()
    if len(iso_plot)>0:
        for p in iso_plot:
            p.remove()

In [56]:
# plotto le isocrone
iso_plot = []
plt.title('CMD con isocrone [E(B-V)={}] [mod_V={}]'.format(ebv,modv))
iso_plot.append(plt.plot(v1good-i1good,v1good,'r-',label='10.5Gyr')[0])
iso_plot.append(plt.plot(v2good-i2good,v2good,'b-',label='11.5Gyr')[0])
iso_plot.append(plt.plot(v3good-i3good,v3good,'g-',label='12.0Gyr')[0])
iso_plot.append(plt.plot(v4good-i4good,v4good,'y-',label='12.5Gyr')[0])
plt.legend()
plt.show()
# Se il grafico viene chiuso bisogna ripartire dalla cella del CMD

In [12]:
# uso interpolatori lineari per ottenere punti da confrontare al cmd mediato
sort1 = np.argsort(v1good)
interp1 = interp.interp1d(v1good[sort1],v1good[sort1]-i1good[sort1],bounds_error=False,fill_value='extrapolate')
v_i_interp1 = interp1(v_mean)

sort2 = np.argsort(v2good)
interp2 = interp.interp1d(v2good[sort2],v2good[sort2]-i2good[sort2],bounds_error=False,fill_value='extrapolate')
v_i_interp2 = interp2(v_mean)

sort3 = np.argsort(v3good)
interp3 = interp.interp1d(v3good[sort3],v3good[sort3]-i3good[sort3],bounds_error=False,fill_value='extrapolate')
v_i_interp3 = interp3(v_mean)

sort4 = np.argsort(v4good)
interp4 = interp.interp1d(v4good[sort4],v4good[sort4]-i4good[sort4],bounds_error=False,fill_value='extrapolate')
v_i_interp4 = interp4(v_mean)

In [13]:
sigma1 = np.linalg.norm(v_i_interp1 - v_i_mean); print '10.5Gyr ->', sigma1
sigma2 = np.linalg.norm(v_i_interp2 - v_i_mean); print '11.5Gyr ->', sigma2
sigma3 = np.linalg.norm(v_i_interp3 - v_i_mean); print '12.0Gyr ->', sigma3
sigma4 = np.linalg.norm(v_i_interp4 - v_i_mean); print '12.5Gyr ->', sigma4

10.5Gyr -> 0.556221996591
11.5Gyr -> 0.453238166776
12.0Gyr -> 0.40888549266
12.5Gyr -> 0.256351998265


### Confronto con isocrona a diversi E(B_V) e diversi mod_V

In [75]:
ages = ['10.5','11.7','12.0','12.5']
index = 0
iso_name = 'iso'+ages[index]+'.dat'
print "Carico l'isocrona {}".format(iso_name)
isocolnames = ['v','i']
iso = np.genfromtxt(iso_name,
                    skip_header=8,
                    usecols=(7,9),
                    names=isocolnames)
v_iso = iso['v']
i_iso = iso['i']

Carico l'isocrona iso10.5.dat


In [88]:
# definisco E(B-V) e modV di riferimento
ebv_ini = 0.35
modv_ini = 12.82

In [125]:
# vario E(B-V) del 10%
# vario modV del 5%
mult_ebv = np.linspace(-0.1 ,0.1, 3, endpoint=True) + 1
ebv_arr = ebv_ini*mult_ebv
mult_modv = np.linspace(-0.1, 0.1 ,3, endpoint=True) + 1
modv_arr = modv_ini*mult_modv

In [127]:
# Questa cella permette di rimuovere TUTTE le isocrone plottate (usare solo dopo aver plottato le isocrone una prima volta)
if 'iso_plot' in locals():
    if len(iso_plot)>0:
        #print iso_plot
        for p in iso_plot:
            try:
                p.remove()
            except:
                print "Impossibile rimuovere, elimino dalla lista."
        iso_plot=[]
plt.legend()

In [126]:
# GLI OCCHI MORIRANNO GUARDANDO LE VARIABILI IN QUESTA CELLA
# tutto in una volta plotto le isocrone e ne calcolo lo scarto dalla madei del cmd
plot_iso = True
iso_plot = []
print "*** Age -> {}Gyr ***".format(ages[index])
for _modv in modv_arr:
    for _ebv in ebv_arr:
        _Av = 3.1 * _ebv
        _Ai = 0.479 * _Av
        _mod0 = _modv - _Av
        _v = v_iso + _mod0 + _Av
        _i = i_iso + _mod0 + _Ai
        _v_i = _v - _i
        if plot_iso:
            _sel = ((_v>12) & (_v<21))
            plt.title("CMD con andamento medio + isocrone {}Gyr ad E(B-V) e modV variabili".format(ages[index]))
            _plot, = plt.plot(_v[_sel]-_i[_sel], _v[_sel],'-',label="E(B-V)={:5.3f} modV={:5.3f}".format(_ebv,_modv))
            iso_plot.append(_plot)
        _sort = np.argsort(_v)
        _interp = interp.interp1d(_v[_sort],_v_i[_sort],bounds_error=False,fill_value='extrapolate')
        _v_i_interp = _interp(v_mean)
        _sigma = np.linalg.norm(_v_i_interp - v_i_mean)
        print "E(B-V)={:4.3f} + modV={:5.3f}  ==> sigma={}".format(_ebv,_modv,_sigma)
plt.legend()
plt.show()

*** Age -> 10.5Gyr ***
E(B-V)=0.315 + modV=11.538  ==> sigma=1.7879048707
E(B-V)=0.350 + modV=11.538  ==> sigma=1.88807033032
E(B-V)=0.385 + modV=11.538  ==> sigma=2.11269106961
E(B-V)=0.315 + modV=12.820  ==> sigma=0.593666630293
E(B-V)=0.350 + modV=12.820  ==> sigma=0.209727239965
E(B-V)=0.385 + modV=12.820  ==> sigma=0.515731713469
E(B-V)=0.315 + modV=14.102  ==> sigma=1.91869255388
E(B-V)=0.350 + modV=14.102  ==> sigma=1.87512667628
E(B-V)=0.385 + modV=14.102  ==> sigma=1.9700932023


# 2) METODO VERTICALE

In [ ]:
# definisco ed uso un cursore per selezionare l'HB
square = Cursor(fig,ax,'r')
square.start()

In [ ]:
x,y = square.get_x_y()
cond = np.where((v_i>x[0]) & (v_i<x[2]) & (v>y[1]) & (v<y[3]))
hb_v_i = v_i[cond]
hb_v = v[cond]

In [ ]:
plt.plot(hb_v_i,hb_v,'m',markersize=6)
plt.show()

In [ ]:
mean_hb_v = np.mean(hb_v)
sigma_hb_v = np.std(hb_v)
print 'Media:', mean_hb_v
print 'Deviazione standard:', sigma_hb_v 

# TEST

In [ ]:
import cursor
reload(cursor)
from cursor import Cursor

In [ ]:
test_cursor = Cursor(fig,ax,color='y',zoom=[1.,2.])

In [ ]:
test_cursor.start()

In [ ]:
ax.set_ylim(10,12)
ax.set_xlim(1,3)
fig.canvas.draw()

In [ ]:
np.savetxt('prova.dat',np.column_stack((xl,yl)))

In [ ]:
cmd_mean_plot.remove()